<a href="https://colab.research.google.com/github/tozanni/Data_Science_Notebooks/blob/main/DL_LSTM_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generación de texto sintético con una red LSTM 

Referencia: https://gist.github.com/maxim5/c35ef2238ae708ccb0e55624e9e0252b


In [1]:
from __future__ import print_function
import numpy as np
import gensim
import string

from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file


In [76]:
## Descargar archivo de abstracts de Stanford

# Original
# https://raw.githubusercontent.com/maxim5/stanford-tensorflow-tutorials/master/data/arxiv_abstracts.txt

# Local
url = 'https://raw.githubusercontent.com/tozanni/Data_Science_Notebooks/main/arxiv_abstracts.txt'
path = get_file('arxiv_abstracts.txt', origin=url)


In [77]:
## Generar sentencias de longitud 40
max_sentence_len = 40
with open(path) as file_:
  docs = file_.readlines()
  sentences = [[word for word in doc.lower().split()[:max_sentence_len]] for doc in docs]

print('Num sentences:', len(sentences))

Num sentences: 7200


### Vectorización con Word2Vec

A continuación se entrenará el modelo de embeddings Word2Vec, dicho modelo nos permitirá representar nuestras palabras en vectores que mantienen ciertas propiedades de similaridad semántica en sus dimensiones.

In [78]:
def word2idx(word):
  return word_model.wv.vocab[word].index

def idx2word(idx):
  return word_model.wv.index2word[idx]

print('Entrenando modelo word2vec con 100 dimensiones...')
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

print('Obtener palabras similares a algunos ejemplos:')
for word in ['model', 'network', 'train', 'learn']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.wv.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))
  

Entrenando word2vec...
Result embedding shape: (1350, 100)
Obtener palabras similares a algunos ejemplos:
  model -> $l_p$ (0.41), technique (0.38), trains (0.34), 2012) (0.33), architecture. (0.33), continuous (0.31), al, (0.30), of (0.30)
  network -> networks (0.33), constrained (0.32), architecture (0.24), there (0.24), trained (0.23), connected (0.23), by (0.23), guide (0.23)
  train -> based (0.37), classical (0.35), eigendecompositions (0.33), sequentially (0.31), map (0.31), extend (0.30), average (0.30), then (0.30)
  learn -> remain (0.37), automatically (0.36), lower (0.36), relevant (0.36), effort (0.36), effectively (0.34), upper (0.34), adopted (0.34)


### Creación de training y test set para LSTM

In [80]:
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)

for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])

print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

# Ejemplo de datos de training y test
# Nuestras secuencias de training y test son los índices 
# de las palabras del diccionario

train_x[0], train_y[0]


train_x shape: (7200, 40)
train_y shape: (7200,)


(array([  4, 463,   5, 464, 465, 130,   1, 104, 131,  26,  19, 170, 466,
         46, 251,  11, 105, 252,  17,   2, 467, 253,   1, 171, 130, 254,
        468, 469, 470, 172, 471, 472,  52, 473, 474, 475, 476,   2, 255,
          0], dtype=int32), 3)

100

In [83]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))

model.add(LSTM(units=16))

#model.add(LSTM(units=emdedding_size))

model.add(Dense(units=vocab_size))
model.add(Activation('softmax')) #El resultado es un vector de probabilidades
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.fit(train_x, train_y,
          batch_size=128,
          epochs=20)


Training LSTM...
Epoch 1/20
57/57 [==============================] - 8s 58ms/step - loss: 6.6431
Epoch 2/20
57/57 [==============================] - 3s 58ms/step - loss: 5.1574
Epoch 3/20
57/57 [==============================] - 3s 57ms/step - loss: 4.2703
Epoch 4/20
57/57 [==============================] - 3s 58ms/step - loss: 3.8740
Epoch 5/20
57/57 [==============================] - 3s 58ms/step - loss: 3.6320
Epoch 6/20
57/57 [==============================] - 3s 58ms/step - loss: 3.4229
Epoch 7/20
57/57 [==============================] - 3s 58ms/step - loss: 3.2232
Epoch 8/20
57/57 [==============================] - 3s 58ms/step - loss: 3.0399
Epoch 9/20
57/57 [==============================] - 3s 57ms/step - loss: 2.8763
Epoch 10/20
57/57 [==============================] - 3s 58ms/step - loss: 2.7215
Epoch 11/20
57/57 [==============================] - 3s 57ms/step - loss: 2.5823
Epoch 12/20
57/57 [==============================] - 3s 57ms/step - loss: 2.4512
Epoch 13/20
57/57 [

In [84]:
def sample(preds, temperature=1.0):
  """
  Metodo de muestreo aleatorio de siguiente palabra.
  Toma como input la distribucion de probabilidad entregada por la red.
  Con cierta proabilidad dependiendo de la temperatura produce la 
  siguiente palabra.
  """

  if temperature <= 0:
    return np.argmax(preds)
  
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    
    #El input se incrementa en cada iteracion a la RNN
    print("Input --> ",word_idxs)
    x=np.array(word_idxs)

    #Tenemos que convertir el input a 3D agregando una dimension dummy
    x=np.expand_dims(x,1) 
    prediction = model.predict(x)
    print("Prediction -->", prediction)

    #No realizar sampling, tomar la palabra con mayor probabilidad
    #idx = np.argmax(prediction[-1])

    #Realizar un muestreo aleatorio
    idx = sample(prediction[-1], temperature=0.7)

    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)


### Generación de texto con el modelo LSTM

Ahora hay que generar texto sintético, en esta etapa utilizaremos el modelo de forma iterativa comenzando por una semilla, posteriormente concatenando el output de cada etapa y pasandolo a la siguiente iteración.

Notar que el modelo entrega una distribución de probabilidad de las siguientes palabras más probables y no es idóneo elegir siempre la mayor (ej. usando argmax) por lo cual se recomienda aplicar un método de sampling sobre dicha distribución.


Referencia de sampling:

https://medium.com/machine-learning-at-petiteprogrammer/sampling-strategies-for-recurrent-neural-networks-9aea02a6616f


In [85]:
next_words = 20 #Cuantas palabras se generaran?

generated_text = generate_next('deep convolutional', next_words)


Input -->  [6, 39]
Prediction --> [[0.01332337 0.00259415 0.00946838 ... 0.00039923 0.00042926 0.00047216]
 [0.00293603 0.00158464 0.00262481 ... 0.000575   0.00059777 0.0006546 ]]
Input -->  [6, 39, 217]
Prediction --> [[0.01332337 0.00259415 0.00946838 ... 0.00039923 0.00042926 0.00047216]
 [0.00293603 0.00158464 0.00262481 ... 0.000575   0.00059777 0.0006546 ]
 [0.00771011 0.0045076  0.00603357 ... 0.00050402 0.00047231 0.00053058]]
Input -->  [6, 39, 217, 1031]
Prediction --> [[0.01332337 0.00259415 0.00946838 ... 0.00039923 0.00042926 0.00047216]
 [0.00293603 0.00158464 0.00262481 ... 0.000575   0.00059777 0.0006546 ]
 [0.0077101  0.0045076  0.00603357 ... 0.00050402 0.00047231 0.00053058]
 [0.00431753 0.00186989 0.00351952 ... 0.00063165 0.00062848 0.00065779]]
Input -->  [6, 39, 217, 1031, 541]
Prediction --> [[0.01332337 0.00259415 0.00946838 ... 0.00039923 0.00042926 0.00047216]
 [0.00293603 0.00158464 0.00262481 ... 0.000575   0.00059777 0.0006546 ]
 [0.0077101  0.0045076  0.

In [86]:
#El texto final generado

generated_text

'deep convolutional algorithms. compiler speeding problems. represent speed methods, weights by speed coupled notion lot which lower-bounded restarting descent) (ii) achievable way'

## Ejercicios

A. Realiza las siguientes modificaciones a la red y comenta los efectos que percibes en a) Valor de pérdida en las épocas. b) Tiempo de entrenamiento.
c) Calidad percibida del texto final generado.

1. Modifica la longitud de las secuencias de input.

2. Modifica la cantidad de unidades de la capa LSTM.

3. Modifica la cantidad de épocas de entrenamiento.

4. Modifica la temperatura de sampling.

B. Presenta 3 ejemplos de texto generado por tu red. Puedes utilizar diferentes palabras de inicialización. 